In [13]:
import csv
import random
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Helper function to generate realistic random data from a given list
def random_data_from_list(data_list):
    return random.choice(data_list)

# Function to generate the CSV data
def generate_csv_data(num_rows, file_name, period_start_dates):
    exits = [f'Exit{i}' for i in range(1, 1001)]
    ports = [f'Port{i}' for i in range(1, 1001)]
    user_groups = [f'UserGroup{i}' for i in range(1, 101)]
    countries = [f'Country{i}' for i in range(1, 21)]

    exit1 = [random_data_from_list(exits) for _ in range(num_rows)]
    port = [random_data_from_list(ports) for _ in range(num_rows)]
    user_group = [random_data_from_list(user_groups) for _ in range(num_rows)]
    country = [random_data_from_list(countries) for _ in range(num_rows)]
    member_id = random.sample(range(1, num_rows + 1), num_rows)

    dates = period_start_dates[:num_rows]
    
    gain_amount_one = np.random.normal(5000, 2000, size=num_rows).astype(int)
    gain_amount_two = np.random.normal(3000, 1500, size=num_rows).astype(int)
    loss_amount = np.random.normal(500, 300, size=num_rows).astype(int)
    
    # Ensure no negative amounts
    gain_amount_one = np.clip(gain_amount_one, 0, None)
    gain_amount_two = np.clip(gain_amount_two, 0, None)
    loss_amount = np.clip(loss_amount, 0, None)

    total_amount = gain_amount_one + gain_amount_two - loss_amount
    total_amount = np.clip(total_amount, 0, None)

    # Prepare the header
    header = [
        'Exit', 
        'Port', 
        'User Group',
        'Country',
        'Member ID', 
        'Date', 
        'Gain Amount One', 
        'Gain Amount Two', 
        'Loss Amount', 
        'Total Amount'
        ]
    
    # Prepare the data
    data = [
        [
            exit1[i], 
            port[i],
            user_group[i],
            country[i],
            member_id[i], 
            dates[i].strftime('%Y-%m-%d'),
            gain_amount_one[i],
            gain_amount_two[i],
            loss_amount[i],
            total_amount[i]
        ]
        for i in range(num_rows)
    ]

    # Sort data by 'Exit' and then by 'Date'
    data.sort(key=lambda x: (x[0], x[5]))  # x[0] is 'Exit', x[5] is 'Date'

    # Write the data to CSV without quotes around strings
    with open(f'{file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=';')
        csvwriter.writerow(header)
        csvwriter.writerows(data)
    
    print(f"CSV file with {num_rows} rows generated!")

# Function to generate the XLSX data
def generate_xlsx_data(num_rows, file_name, period_start_dates):
    exits = [f'Exit{i}' for i in range(1, 1001)]
    exit2 = [random_data_from_list(exits) for _ in range(num_rows)]

    period_start_date = [date.strftime('%d.%m.%Y.') for date in period_start_dates[:num_rows]]

    # Generate random datetime for 'PeriodEndDate' (within 14 days after start_date in the same year)
    period_end_date = []
    for period in period_start_date:
        start_date_for_times = datetime.strptime(period, '%d.%m.%Y.')
        time_base = start_date_for_times + timedelta(days=random.randint(1, 14))  # Ensure times are within 14 days after start_date
        period_end_date.append(time_base.strftime('%d.%m.%Y. %H:%M:%S'))

    gain_amount_three = np.random.normal(10000, 2500, size=num_rows).astype(int)
    gain_amount_three = np.clip(gain_amount_three, 0, None)  # Ensure no negative amounts

    # Create the dataframe
    df_xlsx = pd.DataFrame({
        'PeriodStartDate': period_start_date,
        'Exit': exit2,
        'Gain Amount Three': gain_amount_three,
        'PeriodEndDate': period_end_date
    })

    # Add a row to ensure both files have the same row count
    df_xlsx.loc[num_rows] = ['01.01.2019.', 'Exit1', 1111, '11.01.2019. 11:11:11']

    # Save to XLSX with header on the third row
    with pd.ExcelWriter(f'{file_name}.xlsx', engine='xlsxwriter') as writer:
        df_xlsx.to_excel(writer, index=False, header=True, startrow=2)
    
    print(f"XLSX file with {num_rows} rows generated!")

# Generate common period start dates to be used in both CSV and XLSX files
def generate_common_dates(num_rows):
    start_date = datetime.now() - timedelta(days=365*5)
    return sorted(start_date + timedelta(days=random.randint(0, 365*5)) for _ in range(num_rows))

# Generate files
num_rows_list = [1000, 10000, 100000, 1000000]

for num_rows in num_rows_list:
    common_dates = generate_common_dates(num_rows)
    generate_csv_data(num_rows, f"ModelOne_{num_rows//1000}k_rows", common_dates)
    generate_xlsx_data(num_rows, f"ModelTwo_{num_rows//1000}k_rows", common_dates)

print("Files have been generated successfully!")

CSV file with 1000 rows generated!
XLSX file with 1000 rows generated!
CSV file with 10000 rows generated!
XLSX file with 10000 rows generated!
CSV file with 100000 rows generated!
XLSX file with 100000 rows generated!
CSV file with 1000000 rows generated!
XLSX file with 1000000 rows generated!
Files have been generated successfully!
